In [1]:
import pandas as pd
import numpy as np

import statsmodels.api as sm
import scipy.stats as st
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import metrics
import seaborn as sns
pd.set_option("display.max_columns",None)
# pd.set_option("display.max_rows",None)
from scipy.stats import shapiro,mannwhitneyu,chi2_contingency

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

from IPython.display import Image
# from sklearn.tree import export_graphviz
import imblearn


from numpy.random import seed
from numpy.random import randn
from numpy import mean
from numpy import std


import warnings 
warnings.filterwarnings('ignore')

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn import model_selection
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier,AdaBoostClassifier,GradientBoostingClassifier 
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import RepeatedStratifiedKFold

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report,confusion_matrix

In [3]:
df = pd.read_csv('C:/Users/nabee/Desktop/CIND 820/dffinal.csv')
dfwork = df.copy()

In [4]:
#Splitting Train and Test Data Set
y = dfwork['Revenue']
x = dfwork.drop(['Revenue'], axis = 1)

# checking the shapes
print("Shape of x: ", x.shape)
print("Shape of y: ", y.shape)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size =  0.2, random_state = 0)

print("Shape of x_train :", x_train.shape)
print("Shape of y_train :", y_train.shape)
print("Shape of x_test :", x_test.shape)
print("Shape of y_test :", y_test.shape)

Shape of x:  (12330, 51)
Shape of y:  (12330,)
Shape of x_train : (9864, 51)
Shape of y_train : (9864,)
Shape of x_test : (2466, 51)
Shape of y_test : (2466,)


In [5]:
# Min Max Scaler 
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()    
scaler.fit(x_train)       
x_train = scaler.transform(x_train)    
x_test = scaler.transform(x_test)

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler

In [7]:
from imblearn.over_sampling import BorderlineSMOTE

In [8]:
from imblearn.combine import SMOTEENN
s_enn = SMOTEENN(random_state=42,sampling_strategy = 'auto')
x_train, y_train = s_enn.fit_resample(x_train, y_train.ravel())

print('After OverSampling, the shape of train_X: {}'.format(x_train.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_train.shape))

print("After OverSampling, counts of label '1': {}".format(sum(y_train == 1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_train == 0)))

After OverSampling, the shape of train_X: (14332, 51)
After OverSampling, the shape of train_y: (14332,) 

After OverSampling, counts of label '1': 7571
After OverSampling, counts of label '0': 6761


In [9]:
# fitting models 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb

LR = LogisticRegression().fit(x_train, y_train)
rf = RandomForestClassifier().fit(x_train,y_train)
dt = DecisionTreeClassifier().fit(x_train,y_train)
svc = SVC().fit(x_train,y_train)
ada = AdaBoostClassifier().fit(x_train,y_train)
gbc = GradientBoostingClassifier().fit(x_train,y_train)
gb = xgb.XGBClassifier().fit(x_train,y_train)

In [10]:
# Cross Validation
def cv_comparison(models, X, y, cv = model_selection.RepeatedStratifiedKFold(n_splits=7, n_repeats=3, random_state=1)):
    # Initiate a DataFrame for the averages and a list for all measures
    cv_accuracies = pd.DataFrame()

    # Loop through the models, run a CV, add the average scores to the DataFrame and the scores of 
    # all CVs to the list
    for model in models:
        recalls = cross_val_score(model, x_train, y_train, scoring='recall', cv=cv)
        recall_score = np.mean(recalls).round(2)
        precisions = cross_val_score(model, x_train, y_train, scoring='precision', cv=cv)
        precision_score = np.mean(precisions).round(2)
        accuracies = cross_val_score(model, x_train,y_train, scoring='accuracy', cv=cv)
        accuracy_score = np.mean(accuracies).round(2)
        aucs = cross_val_score(model, x_train,y_train, scoring='roc_auc', cv=cv)
        auc_score = np.mean(aucs).round(2)
        f1s = cross_val_score(model, x_train,y_train, scoring='f1', cv=cv)
        f1_score = np.mean(accuracies).round(2)
        cv_accuracies[str(model).split('(')[0]] = [ accuracy_score, precision_score, recall_score, auc_score, f1_score]
    cv_accuracies.index = ['Accuracy','Precision','Recall',  'AUC','F1']
    result = cv_accuracies.transpose()
    return result
    # return cv_accuracies

In [11]:
LR = LogisticRegression() 
RF = RandomForestClassifier()
DT = DecisionTreeClassifier()
SV = SVC()
AD = AdaBoostClassifier()
GB = GradientBoostingClassifier()
XG = xgb.XGBClassifier()
models = [LR,RF,DT,SV,AD,GB,XG]

In [12]:
import pandas as pd
cv_comparison(models, x_train, y_train)

,Accuracy,Precision,Recall,AUC,F1
LogisticRegression,0.94,0.95,0.93,0.98,0.94
RandomForestClassifier,0.98,0.98,0.98,1.00,0.98
DecisionTreeClassifier,0.96,0.96,0.97,0.96,0.96
SVC,0.96,0.96,0.96,0.99,0.96
AdaBoostClassifier,0.94,0.95,0.94,0.99,0.94
GradientBoostingClassifier,0.96,0.96,0.96,0.99,0.96
XGBClassifier,0.98,0.98,0.99,1.00,0.98


In [13]:
y_pred_lr = LR.predict(x_test)
y_pred_rf = rf.predict(x_test)
y_pred_dt = dt.predict(x_test)
y_pred_svm = svc.predict(x_test)
y_pred_ada = ada.predict(x_test)
y_pred_gbc = gbc.predict(x_test)
y_pred_gb = gb.predict(x_test)

NotFittedError: This LogisticRegression instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:
# cmlr = metrics.confusion_matrix(y_test, y_pred_rf)
# plt.rcParams['figure.figsize'] = (3.5, 3)
# ax = sns.heatmap(cmlr, fmt = 'g',annot = True,cmap="YlGnBu")
# ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels');  
# ax.xaxis.set_ticklabels(['Not Purchased', 'Purchased']); ax.yaxis.set_ticklabels(['Not Purchased', 'Purchased'])
# print ("************************************* Confusion Matrix ******************************************")
# plt.show()

# print ("************************************* Classification Report *************************************")
# crlr = metrics.classification_report(y_test,y_pred_rf)
# print(crlr)

# print ("************************************* description Table *****************************************")
# description = "XG Boost"  #change the name of models
# # cmbase = cmbaselr

# misclassifications = cmlr[0,1] + cmlr[1,0]         #FP + FN
# type1 = cmlr[0,1]
# type2 = cmlr[1,0]
# r1 = (cmlr[1,1]  / (cmlr[1,1] + cmlr[1,0])).round(2)
# r0 = (cmlr[0,0]  / (cmlr[0,0] + cmlr[0,1])).round(2)
# precision = round(precision_score(y_test, y_pred_lr),2)
# recall = round(recall_score(y_test,y_pred_lr),2)
# accuracy = round(accuracy_score(y_test,y_pred_lr),2)
# f1 = round(f1_score(y_test,y_pred_lr),2)
# auc = round(roc_auc_score(y_test,y_pred_lr),2)
# f2 = ((5 * precision * recall)/ (4 * precision + recall)).round(2)
# df_results = pd.DataFrame(np.array([description,misclassifications,accuracy, precision, r1,r0,type1,type2,f1,f2]).reshape(1,-1)
#                                      , columns=['Description','Misclassifications','Accuracy','Precision','Recall Minority','Recall Majority'
#                                                 ,'Type I errors','Type II errors','F1 Score','F2 Score'])
#                                   # ], axis=0)
# df_results